In [1]:
import math
from sqlite3 import connect
import datetime as dt
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
import astropy.units as u
import  astroplan as ap
import pandas as pd


In [2]:
def all_night_observing_session(observer: ap.Observer, utc_date: str) -> [Time, Time]:
    time = Time(utc_date, scale='tdb')
    return [observer.twilight_evening_civil(time, which='nearest'), observer.twilight_morning_civil(time, which='next')]

In [3]:
observer = ap.Observer.at_site("lbt")

observing_sessions = [
    # '2024-01-13',
    # '2024-01-14',
    # '2024-01-15',
    # '2024-01-16',
    # '2024-01-17',
    # '2024-01-18',
    all_night_observing_session(observer, '2024-02-20'),
    # all_night_observing_session(observer, '2024-02-21'),
]


In [4]:
conn = connect("db.sqlite3")

# read base list of targets
targets = pd.read_sql(
    """
        select *
        from tom_target t
        join tom_catalogassociation ca on ca.target_id = t.id and ca.catalog = 'TESS TICv8' and ca.association = 'Primary ID'
        join tom_tess_ticv8 tt on tt.Identifier = ca.catalog_id
        ;""",
    conn,
    index_col="target_id",
)

# get speckle and spectrum data and add it to main targets table
speckle = pd.read_sql("select * from tom_specklerawdata;", conn, index_col="target_id")
spectrum = pd.read_sql(
    "select * from tom_spectrumrawdata;", conn, index_col="target_id"
)

targets = targets.join(speckle.groupby("target_id").id.agg(num_speckle="count")).rename(
    columns={"num_speckle": "Num Speckle"}
)
targets = targets.join(
    spectrum.groupby("target_id").id.agg(num_spectra="count")
).rename(columns={"num_spectra": "Num Spectra"})
targets.fillna(0, inplace=True)
targets["Num Speckle"] = targets["Num Speckle"].astype(int)
targets["Num Spectra"] = targets["Num Spectra"].astype(int)

# add columns for membership in each target list
for (target_list,) in conn.execute("select name from tom_targetlist;").fetchall():
    list_members = [
        result[0]
        for result in conn.execute(
            """
            select t.local_id
            from tom_targetlist tl
            join tom_targetlist_targets tlt on tlt.targetlist_id = tl.id
            join tom_target t on t.id = tlt.target_id
            where tl.name = ?
            ;
            """,
            [target_list],
        ).fetchall()
    ]
    targets[target_list] = False
    targets.loc[targets.local_id.isin(list_members), target_list] = True

# add parameters for the binary systems
params = pd.read_sql(
    """
    select t.local_id, bp.*
    from tom_binaryparameters bp
    join tom_scienceresult sr on sr.id = bp.scienceresult_ptr_id
    join tom_target t on t.id = sr.target_id
    ;""",
    conn,
    index_col="local_id",
)

# TODO: following code only works for binary systems - generalize to n-member systems
joined_params = params.drop(columns=["scienceresult_ptr_id"])
joined_params = joined_params[joined_params["member"] == "A"].join(
    joined_params[joined_params["member"] == "B"],
    on="local_id",
    lsuffix="_A",
    rsuffix="_B",
)
targets = targets.join(joined_params, on="local_id", how="left")

# calculate eclipse phase timings
# TODO: following code only works for binary systems - generalize to n-member systems
# TODO: following code only handles the first observing session - generalize to all sessions
t_beg = observing_sessions[0][0].jd
t_end = observing_sessions[0][1].jd
for (index, row) in targets.iterrows():
    for member in ["A", "B"]:

        if math.isnan(period := row[f"period_{member}"]):
            continue
        targets.loc[index, f"phases_{member}"] = (t_end - t_beg) / period
        for component in ["primary", "secondary"]:
            if math.isnan(t0 := row[f"t0_{component}_{member}"]):
                continue
            t_pre = math.floor((t_beg - t0) / period) * period + t0
            t_post = math.ceil((t_end - t0) / period) * period + t0
            targets.loc[index, f"phase_beg_{member}_{component}"] = (t_beg - t_pre) / period
            targets.loc[index, f"phase_end_{member}_{component}"] = 1 - (t_post - t_end) / period

targets

,id,local_id,source,target_type,id,catalog,catalog_id,association,id,version,...,phases_A,phase_beg_A_primary,phase_end_A_primary,phase_beg_A_secondary,phase_end_A_secondary,phases_B,phase_beg_B_primary,phase_end_B_primary,phase_beg_B_secondary,phase_end_B_secondary
target_id,,,,,,,,,,,,,,,,,,,,,
3366,3366,TIC 270360534,Kostov 2023 arXiv:2309.14200,QuadEB,1024,TESS TICv8,270360534,Primary ID,236,20190415,...,0.015042,0.663424,0.678466,0.231832,0.246874,NaN,NaN,NaN,NaN,NaN
3367,3367,TIC 219469945,Kostov 2022 arXiv:2202.05790,QuadEB,1025,TESS TICv8,219469945,Primary ID,323,20190415,...,0.183863,0.477012,0.660874,0.975611,0.159474,0.033388,0.739076,0.772464,0.256976,0.290364
3368,3368,TIC 20212631,Kostov 2023 arXiv:2309.14200,QuadEB,1026,TESS TICv8,20212631,Primary ID,310,20190415,...,1.681474,0.123358,0.804831,NaN,NaN,0.052715,0.593170,0.645885,NaN,NaN
3369,3369,TIC 150055835,Kostov 2023 arXiv:2309.14200,QuadEB,1027,TESS TICv8,150055835,Primary ID,389,20190415,...,0.157448,0.051421,0.208868,0.551410,0.708857,0.036249,0.057873,0.094122,0.496462,0.532711
3370,3370,TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,1028,TESS TICv8,161043618,Primary ID,215,20190415,...,0.370054,0.701101,0.071155,0.201601,0.571655,0.335684,0.290314,0.625998,0.787414,0.123099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3559,3559,TIC 414969157,Kostov 2022 arXiv:2202.05790,QuadEB,1217,TESS TICv8,414969157,Primary ID,298,20190415,...,0.107907,0.049108,0.157016,0.548508,0.656416,0.072113,0.667706,0.739819,0.195906,0.268019
3560,3560,TIC 27543409,Kostov 2022 arXiv:2202.05790,QuadEB,1218,TESS TICv8,27543409,Primary ID,387,20190415,...,0.235373,0.691975,0.927348,0.195575,0.430948,0.124501,0.960438,0.084939,NaN,NaN
3561,3561,TIC 139914081,Kostov 2023 arXiv:2309.14200,QuadEB,1219,TESS TICv8,139914081,Primary ID,235,20190415,...,0.589840,0.619044,0.208884,NaN,NaN,0.031632,0.314575,0.346206,0.650405,0.682036


In [5]:
#make astroplan objects for each target
fixed_targets = [
    ap.FixedTarget(
        coord=SkyCoord(
            frame="icrs",
            obstime=Time("2000.0", format="jyear", scale="tdb"),
            ra=target["ra"] * u.deg,
            dec=target["dec"] * u.deg,
            pm_ra_cosdec=target["pmRA"] * u.mas / u.yr,
            pm_dec=target["pmDEC"] * u.mas / u.yr,
            # distance=target["plx"] * u.pc,
        ),
        name=target["local_id"],
    )
    for (_, target) in targets.iterrows()
]

constraints = [
    ap.AltitudeConstraint(30*u.deg, 80*u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
    ]

#add a column to table for observability of each target
targets["Observable Nights"] = 0
for observing_session in observing_sessions:
    observable_tonight = ap.is_observable(constraints, observer, fixed_targets, observing_session)
    targets.loc[observable_tonight, "Observable Nights"] += 1


In [6]:
# criteria={
#     # "test": [(targets["HQND"]), ""],
#     "Other Bright with spectra and speckle": [(targets["magnitude"] < 11) & (targets["Num Speckle"] > 0) & (targets["Num Spectra"] > 0) & (targets["HQND"] == False), "Low"],
#     "HQND, Dim": [(targets["HQND"]) & (targets["magnitude"]>13), "Low"],
#     "HQND Bright no spectra": [(targets["HQND"]) & (targets["Num Spectra"] == 0) & (targets["magnitude"] < 13), "Medium"],
#     "HQND Bright with spectra": [(targets["HQND"]) & (targets["Num Spectra"] > 0) & (targets["magnitude"] < 13), "High"],
#     "Featured": [(targets["Featured targets"]), "Highest"],
# }

min_per, max_per = 1.5, 5

criteria = {
    "Bright": [targets["GAIAmag"] < 9, "Lowest"],
    "VATT Test": [(targets["period_A"] > max_per) & (targets["period_B"] < min_per) & (targets["period_B"] > 0)
        | (targets["period_A"] < min_per) & (targets["period_B"] > max_per) & (targets["period_A"] > 0), "Highest"],
}

matching_ids = {}
combined_ids = set()
targets["Priority"] = ""
for name, [criterion, priority] in criteria.items():
    filtered_ids = set(targets[criterion  & (targets["Observable Nights"] > 0)]["local_id"])
    targets.loc[targets.local_id.isin(filtered_ids), "Priority"]=priority
    print(f"{len(filtered_ids):3d} targets from criterion: {name}")
    combined_ids=combined_ids | filtered_ids
    matching_ids[name] = filtered_ids
print(f"Total of {len(combined_ids)} targets")

observing_list = targets[targets["local_id"].isin(combined_ids)]
observing_list[list(observing_list[["local_id", "GAIAmag"]]) + list(observing_list.loc[:, "phases_A":])].sort_values("GAIAmag")

  4 targets from criterion: Bright
  9 targets from criterion: VATT Test
Total of 13 targets


,local_id,GAIAmag,phases_A,phase_beg_A_primary,phase_end_A_primary,phase_beg_A_secondary,phase_end_A_secondary,phases_B,phase_beg_B_primary,phase_end_B_primary,phase_beg_B_secondary,phase_end_B_secondary,Observable Nights,Priority
target_id,,,,,,,,,,,,,,
3398,TIC 344541836,7.86518,0.207336,0.306361,0.513697,0.653461,0.860796,0.181348,0.845357,0.026706,0.344357,0.525706,1,Lowest
3375,TIC 367448265,7.87389,1.194691,0.327729,0.522420,NaN,NaN,0.267842,0.491244,0.759086,0.986844,0.254686,1,Lowest
3439,TIC 153406662,7.92700,0.198177,0.637952,0.836129,0.142333,0.340509,0.056376,0.936963,0.993339,NaN,NaN,1,Lowest
3406,TIC 266395331,8.75267,0.163880,0.279007,0.442887,0.780085,0.943965,0.095364,0.985366,0.080730,NaN,NaN,1,Lowest
3449,TIC 238558210,9.25867,0.513854,0.937790,0.451644,NaN,NaN,0.093611,0.797499,0.891110,0.281886,0.375497,1,Highest
3385,TIC 470710327,9.52218,0.452314,0.209454,0.661768,0.706954,0.159268,0.025045,0.946802,0.971847,NaN,NaN,1,Highest
3368,TIC 20212631,10.51590,1.681474,0.123358,0.804831,NaN,NaN,0.052715,0.593170,0.645885,NaN,NaN,1,Highest
3466,TIC 283940788,11.72660,0.569830,0.768257,0.338086,0.274257,0.844087,0.061174,0.788003,0.849178,0.483603,0.544778,1,Highest
3419,TIC 24700485,12.52720,0.675396,0.875812,0.551209,0.382902,0.058299,0.046819,0.621250,0.668069,NaN,NaN,1,Highest


In [7]:
#exposure data from Robbie's LBT observing readme, with dupes removed manually
import numpy as np
mag=np.array([7.9,9.4,9.5,10,10.2,10.3,10.6,10.7,10.8,10.9,11,11.1,11.2,11.3,11.4,11.5,11.7,11.8,11.9,12,12.1,12.3,12.4,12.5,12.6,12.7,12.8,13.2,13.3,13.4,13.5,13.6,13.7,13.8,13.9,])
sec=np.array([60,60,60,60,60,60,60,60,60,72,72,84,78,84,102,102,132,138,150,192,180,240,270,258,270,318,360,510,594,630,666,726,768,858,918,])

from scipy.interpolate import CubicSpline
cs = CubicSpline(mag, sec)

#print out results in LBT readme format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes")
print("-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["GAIAmag"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f"{target["local_id"]:<15s} "
        f"{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    "
        f"{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      "
        f"{target["GAIAmag"]:5.2f}    "
        f"{target["pmRA"]:6.2f}   "
        f"{target["pmDEC"]:6.2f}     "
        f"F300/CDII/CDVI     "
        f"1x{exposure:4.0f}sec     "
        f"{readout_sec:5.0f} sec = {readout_min:4.1f}min   "
        f"{target["Priority"]:<7s}   "
        "--"
    )

TargetName       RA (J2000)     DEC (J2000)       Gmag     pmRA     pmDEC     Mode               ExposureTime   ExecutionTime        Priority  Notes
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
TIC 283940788    0:35:24.34    +62:54:05.74      11.73     -2.73    -0.15     F300/CDII/CDVI     1x 135sec       275 sec =  4.6min   Highest   --
TIC 266395331    4:03:31.90    +48:33:51.24       8.75     21.04   -28.05     F300/CDII/CDVI     1x  60sec       200 sec =  3.3min   Lowest    --
TIC 459959916    4:45:57.52     +4:49:46.63      13.23      2.92     5.88     F300/CDII/CDVI     1x 536sec       676 sec = 11.3min   Highest   --
TIC 13021681     5:02:08.35    -24:03:41.91      14.77     -2.86     4.03     F300/CDII/CDVI     1x-18787sec     -18647 sec = -310.8min   Highest   --
TIC 367448265    5:13:31.79    +35:39:10.99       7.87     -5.86    -3.43   

In [8]:
#output targets in LBT obs.txt format
#NOTE: targets are printed in ascending RA.  Rotate the list so first observable target is first.
print("#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp")
for _,target in targets[targets["local_id"].isin(combined_ids)].sort_values("ra").iterrows():
    exposure = cs(target["GAIAmag"])
    readout_sec = exposure + 140
    readout_min = readout_sec / 60
    print(
        f"{target["local_id"]:<15s}  "
        f"{Angle(target["ra"]*u.deg).to_string(unit=u.hour,decimal=False,precision=2,sep=":"):>11s}    "
        f"{Angle(target["dec"]*u.deg).to_string(unit=u.deg,decimal=False,precision=2,sep=":",alwayssign=True):>12s}      "
        f"{target["GAIAmag"]:5.2f}     "
        f"F300/CDII/CDVI     "
        f"1x{exposure:4.0f}sec     "
    )

#TargetName      RA(J2000)      DEC(J2000)        VMag      MODE               t_exp
TIC 283940788     0:35:24.34    +62:54:05.74      11.73     F300/CDII/CDVI     1x 135sec     
TIC 266395331     4:03:31.90    +48:33:51.24       8.75     F300/CDII/CDVI     1x  60sec     
TIC 459959916     4:45:57.52     +4:49:46.63      13.23     F300/CDII/CDVI     1x 536sec     
TIC 13021681      5:02:08.35    -24:03:41.91      14.77     F300/CDII/CDVI     1x-18787sec     
TIC 367448265     5:13:31.79    +35:39:10.99       7.87     F300/CDII/CDVI     1x  60sec     
TIC 24700485      5:24:49.11     -7:35:32.49      12.53     F300/CDII/CDVI     1x 257sec     
TIC 238558210     6:04:21.90    +20:32:03.26       9.26     F300/CDII/CDVI     1x  60sec     
TIC 153406662     6:12:54.95     +9:02:08.86       7.93     F300/CDII/CDVI     1x  60sec     
TIC 36439758      6:48:56.43     -0:27:52.97      13.03     F300/CDII/CDVI     1x 418sec     
TIC 20212631     15:08:09.13    +39:58:12.86      10.52     F300/CD